## **Setting up the Environment**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! rm -rf indic_nlp_resources
! git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git

Cloning into 'indic_nlp_resources'...
remote: Enumerating objects: 139, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 139 (delta 2), reused 2 (delta 0), pack-reused 126
Receiving objects: 100% (139/139), 149.77 MiB | 25.87 MiB/s, done.
Resolving deltas: 100% (53/53), done.


In [ ]:
% cd /content/drive/MyDrive/Colab Notebooks/Hate-Speech-Detection-in-Tamil/HASOC Tamil/Task 1

/content/drive/MyDrive/Colab Notebooks/Hate-Speech-Detection-in-Tamil/HASOC Tamil/Task 1


In [ ]:
! pip install -U -q indic-nlp-library

     |████████████████████████████████| 40 kB 4.5 MB/s 
     |████████████████████████████████| 2.8 MB 28.9 MB/s 


## **Importing Libraries**

In [ ]:
import pandas as pd

import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

from indicnlp import common
from indicnlp import loader
from indicnlp.morph import unsupervised_morph

## **Setting up Stemmer**

In [ ]:
INDIC_NLP_RESOURCES = "./indic_nlp_resources"
common.set_resources_path(INDIC_NLP_RESOURCES)
loader.load()
analyzer = unsupervised_morph.UnsupervisedMorphAnalyzer('ta')

## **Loading Data**

In [ ]:
dataframe = pd.read_csv("Dataset/tamil_offensive_train.tsv", sep='\t')

In [ ]:
dataframe.head()

,id,text,category
0,tam1,திருமலை நாயக்கர் பேரவை சார்பாக படம் வெற்றி பெற...,NOT
1,tam2,இந்த ட்ரெய்லர் கூட பார்க்கிற மாதிரி இல்லை.. இத...,OFF
2,tam3,மைசூரு செட்டியார் சமூகத்தின் சார்பாக இப்படம் வ...,NOT
3,tam4,மொத்த சாதியும் ஒரு சாதிக்கு எதிரா நிக்குது.......,OFF
4,tam5,only for விஜய் சேதுபதி and STR,NOT


In [ ]:
dataframe.describe()

,id,text,category
count,5880,5880,5880
unique,5880,5812,3
top,tam1,வந்தா ராஜாவா தான் வருவேன்........ ராஜாவா வந்தா...,NOT
freq,1,5,4724


In [ ]:
dataframe['category'].unique()

array(['NOT', 'OFF', 'not-Tamil'], dtype=object)

In [ ]:
dataframe.shape

(5880, 3)

In [ ]:
dataframe = dataframe.loc[dataframe['category'] != 'not-Tamil']
dataframe['category'].unique()

array(['NOT', 'OFF'], dtype=object)

In [ ]:
dataframe.shape

(5877, 3)

In [ ]:
text = dataframe['text']

In [ ]:
text

0       திருமலை நாயக்கர் பேரவை சார்பாக படம் வெற்றி பெற...
1       இந்த ட்ரெய்லர் கூட பார்க்கிற மாதிரி இல்லை.. இத...
2       மைசூரு செட்டியார் சமூகத்தின் சார்பாக இப்படம் வ...
3       மொத்த சாதியும் ஒரு சாதிக்கு எதிரா நிக்குது.......
4                          only for விஜய் சேதுபதி and STR
                              ...                        
5875    இப்போதான் ஜாதி பிரச்சினை இல்லாம இருக்கு இப்போ ...
5876    இது ஒரு சாதி ரீதியான படம் இல்லை தான் செல்லும் ...
5877    கதை தெரியவில்லை  இருந்தாலும் மாறுபட்ட சிந்தனை ...
5878    அட பரவால்லையே 8.5 டிஸ்லைக்ஸ் போட்ருக்கானுங்க.....
5879    உதயநிதிக்கு நடிப்பு வரலனு சொல்ரவங்க லைக் போடுங...
Name: text, Length: 5877, dtype: object

In [ ]:
label = dataframe['category']

In [ ]:
label

0       NOT
1       OFF
2       NOT
3       OFF
4       NOT
       ... 
5875    NOT
5876    NOT
5877    NOT
5878    OFF
5879    NOT
Name: category, Length: 5877, dtype: object

## **Data Preprocessing**

In [ ]:
le = LabelEncoder()
label = le.fit_transform(label)
label

array([0, 1, 0, ..., 0, 1, 0])

In [ ]:
text = text.str.replace(r"[+/#@&*$%:]",'')
text = text.to_numpy()
text

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


array(['திருமலை நாயக்கர் பேரவை சார்பாக படம் வெற்றி பெற வாழ்த்துக்கள்',
       'இந்த ட்ரெய்லர் கூட பார்க்கிற மாதிரி இல்லை.. இதை தியேட்டர் ல போய் பார்க்கனுமா.. .',
       'மைசூரு செட்டியார் சமூகத்தின் சார்பாக இப்படம் வெற்றிபெற வாழ்த்துகிறேன்',
       ...,
       'கதை தெரியவில்லை  இருந்தாலும் மாறுபட்ட சிந்தனை வாழ்த்துக்கள்  பறையர் சார்பாக  நன்றி மோகன் ஜி',
       'அட பரவால்லையே 8.5 டிஸ்லைக்ஸ் போட்ருக்கானுங்க.. பரவால வயிறு எறிஞ்சி சாவுங்கடா சாவுங்கடா சாக்காடை நாய்ங்களா..இன்னும் நிறைய டிஸ்லைக்ஸ் எதிர் பாக்குறன்..',
       'உதயநிதிக்கு நடிப்பு வரலனு சொல்ரவங்க லைக் போடுங்கள் ...'],
      dtype=object)

In [ ]:
text = text.tolist()
stemmed_text = []

for line in text:
    line = line.split()
    stemmed_line = ''
    for token in line:
        token = analyzer.morph_analyze_document(token.split(' '))[0]
        stemmed_line += token + ' '
    stemmed_text.append(stemmed_line)

text = np.array(stemmed_text)

## **Feature Extraction**

In [ ]:
vectorizer = TfidfVectorizer(min_df = 5)
X = vectorizer.fit_transform(text)
X = X.todense()

In [ ]:
X.shape

(5877, 689)

In [ ]:
y = label
y.shape

(5877,)

## **Train Test Split**

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train.shape

(4701, 689)

In [ ]:
X_val.shape

(1176, 689)

## **Logistic Regression**

In [ ]:
lr = LogisticRegression(C=10, max_iter=1000, penalty='none', solver='sag')
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_val)
print(classification_report(y_val, lr_pred))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:1484: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio parameters"
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


              precision    recall  f1-score   support

           0       0.86      0.92      0.89       950
           1       0.53      0.40      0.46       226

    accuracy                           0.82      1176
   macro avg       0.70      0.66      0.67      1176
weighted avg       0.80      0.82      0.81      1176



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


## **Support Vector Machine**

In [ ]:
svm = SVC(C=0.1, decision_function_shape='ovo', max_iter=1000)
svm.fit(X_train, y_train)
svm_pred = svm.predict(X_val)
print(classification_report(y_val, svm_pred))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


              precision    recall  f1-score   support

           0       0.81      1.00      0.89       950
           1       0.00      0.00      0.00       226

    accuracy                           0.81      1176
   macro avg       0.40      0.50      0.45      1176
weighted avg       0.65      0.81      0.72      1176



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## **Naive Bayes**

In [ ]:
nb = GaussianNB()
nb.fit(X_train, y_train)
nb_pred = nb.predict(X_val)
print(classification_report(y_val, nb_pred))

              precision    recall  f1-score   support

           0       0.90      0.23      0.36       950
           1       0.22      0.89      0.35       226

    accuracy                           0.36      1176
   macro avg       0.56      0.56      0.36      1176
weighted avg       0.77      0.36      0.36      1176



/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


## **Stochastic Gradient Descent**

In [ ]:
sgd = SGDClassifier(alpha=1e-05, loss='perceptron', penalty='elasticnet')
sgd.fit(X_train, y_train)
sgd_pred = sgd.predict(X_val)
print(classification_report(y_val, sgd_pred))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


              precision    recall  f1-score   support

           0       0.82      0.98      0.90       950
           1       0.63      0.12      0.20       226

    accuracy                           0.82      1176
   macro avg       0.73      0.55      0.55      1176
weighted avg       0.79      0.82      0.76      1176



/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


## **K Nearest Neighbours**

In [ ]:
knn = KNeighborsClassifier(p=1, weights='distance')
knn.fit(X_train, y_train)
knn_pred = knn.predict(X_val)
print(classification_report(y_val, knn_pred))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


              precision    recall  f1-score   support

           0       0.82      0.98      0.89       950
           1       0.50      0.08      0.14       226

    accuracy                           0.81      1176
   macro avg       0.66      0.53      0.52      1176
weighted avg       0.76      0.81      0.75      1176



## **Decision Tree**

In [ ]:
dt = DecisionTreeClassifier(criterion='entropy', max_features='auto')
dt.fit(X_train, y_train)
dt_pred = dt.predict(X_val)
print(classification_report(y_val, dt_pred))

              precision    recall  f1-score   support

           0       0.84      0.88      0.86       950
           1       0.38      0.31      0.34       226

    accuracy                           0.77      1176
   macro avg       0.61      0.59      0.60      1176
weighted avg       0.75      0.77      0.76      1176



/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


## **Random Forest**

In [ ]:
rf = RandomForestClassifier(criterion='entropy', max_features='sqrt', n_estimators=200)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_val)
print(classification_report(y_val, rf_pred))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


              precision    recall  f1-score   support

           0       0.82      0.98      0.90       950
           1       0.59      0.10      0.17       226

    accuracy                           0.81      1176
   macro avg       0.71      0.54      0.53      1176
weighted avg       0.78      0.81      0.76      1176



/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


## **Majority Voting**

In [ ]:
mv_pred = []

for i in range(len(lr_pred)):
    one = 0
    zero = 0
    # Removing KNN and NB from predictions due to poor performance
    predictions = [lr_pred[i], knn_pred[i], rf_pred[i]]
    for pred in predictions:
        if pred == 1: one += 1
        if pred == 0: zero +=1
    if one > zero: mv_pred.append(1)
    else: mv_pred.append(0)

mv_pred = np.array(mv_pred)

print(classification_report(y_val, mv_pred))

              precision    recall  f1-score   support

           0       0.82      0.99      0.90       950
           1       0.62      0.10      0.17       226

    accuracy                           0.82      1176
   macro avg       0.72      0.54      0.54      1176
weighted avg       0.78      0.82      0.76      1176



## **Loading Testing Data**

In [ ]:
dataframe_test = pd.read_csv(
    "Dataset/tam_offesive_withoutlabels_test.tsv", 
    sep='\t'
)
dataframe_test.head()

,id,text
0,tam_test_1,தலைவா STR இதுக்குதான் கதுருந்தோம் மாஸ் தலைவா t...
1,tam_test_2,நாயுடு மக்கள் சார்பாக திரைப்படம் வெற்றி பெற வா...
2,tam_test_3,தில்லானா முயற்சி தஞ்சை கோனார்
3,tam_test_4,திரௌபதியின் துகிலுறித்த போது காத்த கண்ணனின் யா...
4,tam_test_5,நான் தியேட்டர்லே படம் பார்த்து 35 வருஷமாச்சு! ...


In [ ]:
dataframe_test.describe()

,id,text
count,654,654
unique,654,654
top,tam_test_1,தலைவா STR இதுக்குதான் கதுருந்தோம் மாஸ் தலைவா t...
freq,1,1


In [ ]:
text_test = dataframe_test['text']

## **Test Data Preprocessing**

In [ ]:
text_test = text_test.str.replace(r"[+/#@&*$%:]",'')
text_test = text_test.to_numpy()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [ ]:
text_test = text_test.tolist()
stemmed_text = []

for line in text_test:
    line = line.split()
    stemmed_line = ''
    for token in line:
        token = analyzer.morph_analyze_document(token.split(' '))[0]
        stemmed_line += token + ' '
    stemmed_text.append(stemmed_line)

text_test = np.array(stemmed_text)

## **Test Feature Extraction**

In [ ]:
vectorizer = TfidfVectorizer(min_df = 5)
X_test = vectorizer.fit_transform(text_test)
X_test = X_test.todense()

In [ ]:
X_test.shape

(654, 218)

In [ ]:
X_test = X_test.tolist()

In [ ]:
for i in range(len(X_test)):
    for n in range(471):
        X_test[i].append(0)

In [ ]:
X_test = np.array(X_test)
X_test.shape

(654, 689)

## **Getting Predictions**

In [ ]:
# Logistic Regression
lr_test_pred = lr.predict(X_test)

# Support Vector Machine
svm_test_pred = svm.predict(X_test)

# Naive Bayes
nb_test_pred = nb.predict(X_test)

# Stochastic Gradient Descent
sgd_test_pred = sgd.predict(X_test)

# K Nearest Neighbours
knn_test_pred = knn.predict(X_test)

# Decision Tree
dt_test_pred = dt.predict(X_test)

# Random Forest
rf_test_pred = rf.predict(X_test)

In [ ]:
# Majority Voting
mv_test_pred = []

for i in range(len(lr_test_pred)):
    one = 0
    zero = 0
    predictions = [lr_test_pred[i], svm_test_pred[i], sgd_test_pred[i], 
                   dt_test_pred[i], rf_test_pred[i]]
    for pred in predictions:
        if pred == 1: one += 1
        if pred == 0: zero +=1
    if one > zero: mv_test_pred.append(1)
    else: mv_test_pred.append(0)

mv_test_pred = np.array(mv_test_pred)

## **Evaluating the Predictions**

In [ ]:
dataframe_eval = pd.read_csv(
    "Dataset/tamil_offensive_test task 1 with labels.tsv", 
    sep='\t'
)

In [ ]:
y_test = dataframe_eval['category']
y_test = LabelEncoder().fit_transform(y_test)

In [ ]:
print('Logistic Regression')
print(classification_report(y_test, lr_test_pred))

Logistic Regression
              precision    recall  f1-score   support

           0       0.82      0.82      0.82       536
           1       0.17      0.16      0.16       118

    accuracy                           0.70       654
   macro avg       0.49      0.49      0.49       654
weighted avg       0.70      0.70      0.70       654



In [ ]:
print('Support Vector Machine')
print(classification_report(y_test, svm_test_pred))

Support Vector Machine
              precision    recall  f1-score   support

           0       0.82      1.00      0.90       536
           1       0.00      0.00      0.00       118

    accuracy                           0.82       654
   macro avg       0.41      0.50      0.45       654
weighted avg       0.67      0.82      0.74       654



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print('Naive Bayes')
print(classification_report(y_test, nb_test_pred))

Naive Bayes
              precision    recall  f1-score   support

           0       0.81      0.71      0.76       536
           1       0.15      0.24      0.19       118

    accuracy                           0.62       654
   macro avg       0.48      0.47      0.47       654
weighted avg       0.69      0.62      0.65       654



In [ ]:
print('Stochastic Gradient Descent')
print(classification_report(y_test, sgd_test_pred))

Stochastic Gradient Descent
              precision    recall  f1-score   support

           0       0.82      0.93      0.87       536
           1       0.15      0.06      0.08       118

    accuracy                           0.77       654
   macro avg       0.48      0.49      0.48       654
weighted avg       0.70      0.77      0.73       654



In [ ]:
print('K Nearest Neighbours')
print(classification_report(y_test, knn_test_pred))

K Nearest Neighbours
              precision    recall  f1-score   support

           0       0.82      0.99      0.90       536
           1       0.25      0.01      0.02       118

    accuracy                           0.82       654
   macro avg       0.53      0.50      0.46       654
weighted avg       0.72      0.82      0.74       654



In [ ]:
print('Decision Tree')
print(classification_report(y_test, dt_test_pred))

Decision Tree
              precision    recall  f1-score   support

           0       0.83      0.67      0.74       536
           1       0.20      0.36      0.26       118

    accuracy                           0.62       654
   macro avg       0.51      0.52      0.50       654
weighted avg       0.71      0.62      0.65       654



In [ ]:
print('Random Forest')
print(classification_report(y_test, rf_test_pred))

Random Forest
              precision    recall  f1-score   support

           0       0.83      0.96      0.89       536
           1       0.30      0.08      0.13       118

    accuracy                           0.80       654
   macro avg       0.56      0.52      0.51       654
weighted avg       0.73      0.80      0.75       654



In [ ]:
print('Majority Voting')
print(classification_report(y_test, mv_test_pred))

Majority Voting
              precision    recall  f1-score   support

           0       0.82      0.95      0.88       536
           1       0.19      0.05      0.08       118

    accuracy                           0.79       654
   macro avg       0.51      0.50      0.48       654
weighted avg       0.71      0.79      0.74       654



## **Saving the Predictions**

In [ ]:
# Logistic Regression

predictions = le.inverse_transform(lr_test_pred)

dataframe_test_lr = {
    'id': dataframe_test['id'],
    'text': dataframe_test['text'],
    'category': predictions
}

dataframe_test_lr = pd.DataFrame(dataframe_test_lr)
dataframe_test_lr.to_csv("Output/TF-IDF-LogisticRegression.csv")
dataframe_test_lr

,id,text,category
0,tam_test_1,தலைவா STR இதுக்குதான் கதுருந்தோம் மாஸ் தலைவா t...,NOT
1,tam_test_2,நாயுடு மக்கள் சார்பாக திரைப்படம் வெற்றி பெற வா...,OFF
2,tam_test_3,தில்லானா முயற்சி தஞ்சை கோனார்,NOT
3,tam_test_4,திரௌபதியின் துகிலுறித்த போது காத்த கண்ணனின் யா...,NOT
4,tam_test_5,நான் தியேட்டர்லே படம் பார்த்து 35 வருஷமாச்சு! ...,NOT
...,...,...,...
649,tam_test_650,நான் தலைவரோட ரசிகன்னு சொல்லிடு தலைவரை ஒடசல் போ...,NOT
650,tam_test_651,"வாழ்த்துக்கள்,உண்மை கதை உலகத்துக்கு காட்டிய மோ...",NOT
651,tam_test_652,குருமா and பா..குஞ்சித் பயலுக கதர்ரது இன்னும் ...,NOT
652,tam_test_653,வெள்ளார்கள் சார்பாக படம் வெற்றி பெற வாழ்த்துக்...,OFF


In [ ]:
# Support Vector Machine

predictions = le.inverse_transform(svm_test_pred)

dataframe_test_svm = {
    'id': dataframe_test['id'],
    'text': dataframe_test['text'],
    'category': predictions
}

dataframe_test_svm = pd.DataFrame(dataframe_test_svm)
dataframe_test_svm.to_csv("Output/TF-IDF-SupportVectorMachine.csv")
dataframe_test_svm

,id,text,category
0,tam_test_1,தலைவா STR இதுக்குதான் கதுருந்தோம் மாஸ் தலைவா t...,NOT
1,tam_test_2,நாயுடு மக்கள் சார்பாக திரைப்படம் வெற்றி பெற வா...,NOT
2,tam_test_3,தில்லானா முயற்சி தஞ்சை கோனார்,NOT
3,tam_test_4,திரௌபதியின் துகிலுறித்த போது காத்த கண்ணனின் யா...,NOT
4,tam_test_5,நான் தியேட்டர்லே படம் பார்த்து 35 வருஷமாச்சு! ...,NOT
...,...,...,...
649,tam_test_650,நான் தலைவரோட ரசிகன்னு சொல்லிடு தலைவரை ஒடசல் போ...,NOT
650,tam_test_651,"வாழ்த்துக்கள்,உண்மை கதை உலகத்துக்கு காட்டிய மோ...",NOT
651,tam_test_652,குருமா and பா..குஞ்சித் பயலுக கதர்ரது இன்னும் ...,NOT
652,tam_test_653,வெள்ளார்கள் சார்பாக படம் வெற்றி பெற வாழ்த்துக்...,NOT


In [ ]:
# Naive Bayes

predictions = le.inverse_transform(nb_test_pred)

dataframe_test_nb = {
    'id': dataframe_test['id'],
    'text': dataframe_test['text'],
    'category': predictions
}

dataframe_test_nb = pd.DataFrame(dataframe_test_nb)
dataframe_test_nb.to_csv("Output/TF-IDF-NaiveBayes.csv")
dataframe_test_nb

,id,text,category
0,tam_test_1,தலைவா STR இதுக்குதான் கதுருந்தோம் மாஸ் தலைவா t...,OFF
1,tam_test_2,நாயுடு மக்கள் சார்பாக திரைப்படம் வெற்றி பெற வா...,OFF
2,tam_test_3,தில்லானா முயற்சி தஞ்சை கோனார்,OFF
3,tam_test_4,திரௌபதியின் துகிலுறித்த போது காத்த கண்ணனின் யா...,NOT
4,tam_test_5,நான் தியேட்டர்லே படம் பார்த்து 35 வருஷமாச்சு! ...,OFF
...,...,...,...
649,tam_test_650,நான் தலைவரோட ரசிகன்னு சொல்லிடு தலைவரை ஒடசல் போ...,NOT
650,tam_test_651,"வாழ்த்துக்கள்,உண்மை கதை உலகத்துக்கு காட்டிய மோ...",NOT
651,tam_test_652,குருமா and பா..குஞ்சித் பயலுக கதர்ரது இன்னும் ...,NOT
652,tam_test_653,வெள்ளார்கள் சார்பாக படம் வெற்றி பெற வாழ்த்துக்...,OFF


In [ ]:
# Stochastic Gradient Descent

predictions = le.inverse_transform(sgd_test_pred)

dataframe_test_sgd = {
    'id': dataframe_test['id'],
    'text': dataframe_test['text'],
    'category': predictions
}

dataframe_test_sgd = pd.DataFrame(dataframe_test_sgd)
dataframe_test_sgd.to_csv("Output/TF-IDF-StochasticGradientDescent.csv")
dataframe_test_sgd

,id,text,category
0,tam_test_1,தலைவா STR இதுக்குதான் கதுருந்தோம் மாஸ் தலைவா t...,NOT
1,tam_test_2,நாயுடு மக்கள் சார்பாக திரைப்படம் வெற்றி பெற வா...,NOT
2,tam_test_3,தில்லானா முயற்சி தஞ்சை கோனார்,NOT
3,tam_test_4,திரௌபதியின் துகிலுறித்த போது காத்த கண்ணனின் யா...,NOT
4,tam_test_5,நான் தியேட்டர்லே படம் பார்த்து 35 வருஷமாச்சு! ...,NOT
...,...,...,...
649,tam_test_650,நான் தலைவரோட ரசிகன்னு சொல்லிடு தலைவரை ஒடசல் போ...,NOT
650,tam_test_651,"வாழ்த்துக்கள்,உண்மை கதை உலகத்துக்கு காட்டிய மோ...",NOT
651,tam_test_652,குருமா and பா..குஞ்சித் பயலுக கதர்ரது இன்னும் ...,NOT
652,tam_test_653,வெள்ளார்கள் சார்பாக படம் வெற்றி பெற வாழ்த்துக்...,OFF


In [ ]:
# K Nearest Neighbours

predictions = le.inverse_transform(knn_test_pred)

dataframe_test_knn = {
    'id': dataframe_test['id'],
    'text': dataframe_test['text'],
    'category': predictions
}

dataframe_test_knn = pd.DataFrame(dataframe_test_knn)
dataframe_test_knn.to_csv("Output/TF-IDF-KNearestNeighbours.csv")
dataframe_test_knn

,id,text,category
0,tam_test_1,தலைவா STR இதுக்குதான் கதுருந்தோம் மாஸ் தலைவா t...,NOT
1,tam_test_2,நாயுடு மக்கள் சார்பாக திரைப்படம் வெற்றி பெற வா...,NOT
2,tam_test_3,தில்லானா முயற்சி தஞ்சை கோனார்,NOT
3,tam_test_4,திரௌபதியின் துகிலுறித்த போது காத்த கண்ணனின் யா...,NOT
4,tam_test_5,நான் தியேட்டர்லே படம் பார்த்து 35 வருஷமாச்சு! ...,NOT
...,...,...,...
649,tam_test_650,நான் தலைவரோட ரசிகன்னு சொல்லிடு தலைவரை ஒடசல் போ...,NOT
650,tam_test_651,"வாழ்த்துக்கள்,உண்மை கதை உலகத்துக்கு காட்டிய மோ...",NOT
651,tam_test_652,குருமா and பா..குஞ்சித் பயலுக கதர்ரது இன்னும் ...,NOT
652,tam_test_653,வெள்ளார்கள் சார்பாக படம் வெற்றி பெற வாழ்த்துக்...,NOT


In [ ]:
# Decision Tree

predictions = le.inverse_transform(dt_test_pred)

dataframe_test_dt = {
    'id': dataframe_test['id'],
    'text': dataframe_test['text'],
    'category': predictions
}

dataframe_test_dt = pd.DataFrame(dataframe_test_dt)
dataframe_test_dt.to_csv("Output/TF-IDF-DecisionTree.csv")
dataframe_test_dt

,id,text,category
0,tam_test_1,தலைவா STR இதுக்குதான் கதுருந்தோம் மாஸ் தலைவா t...,NOT
1,tam_test_2,நாயுடு மக்கள் சார்பாக திரைப்படம் வெற்றி பெற வா...,OFF
2,tam_test_3,தில்லானா முயற்சி தஞ்சை கோனார்,NOT
3,tam_test_4,திரௌபதியின் துகிலுறித்த போது காத்த கண்ணனின் யா...,NOT
4,tam_test_5,நான் தியேட்டர்லே படம் பார்த்து 35 வருஷமாச்சு! ...,NOT
...,...,...,...
649,tam_test_650,நான் தலைவரோட ரசிகன்னு சொல்லிடு தலைவரை ஒடசல் போ...,NOT
650,tam_test_651,"வாழ்த்துக்கள்,உண்மை கதை உலகத்துக்கு காட்டிய மோ...",OFF
651,tam_test_652,குருமா and பா..குஞ்சித் பயலுக கதர்ரது இன்னும் ...,NOT
652,tam_test_653,வெள்ளார்கள் சார்பாக படம் வெற்றி பெற வாழ்த்துக்...,OFF


In [ ]:
# Random Forest

predictions = le.inverse_transform(rf_test_pred)

dataframe_test_rf = {
    'id': dataframe_test['id'],
    'text': dataframe_test['text'],
    'category': predictions
}

dataframe_test_rf = pd.DataFrame(dataframe_test_rf)
dataframe_test_rf.to_csv("Output/TF-IDF-RandomForest.csv")
dataframe_test_rf

,id,text,category
0,tam_test_1,தலைவா STR இதுக்குதான் கதுருந்தோம் மாஸ் தலைவா t...,NOT
1,tam_test_2,நாயுடு மக்கள் சார்பாக திரைப்படம் வெற்றி பெற வா...,NOT
2,tam_test_3,தில்லானா முயற்சி தஞ்சை கோனார்,NOT
3,tam_test_4,திரௌபதியின் துகிலுறித்த போது காத்த கண்ணனின் யா...,NOT
4,tam_test_5,நான் தியேட்டர்லே படம் பார்த்து 35 வருஷமாச்சு! ...,NOT
...,...,...,...
649,tam_test_650,நான் தலைவரோட ரசிகன்னு சொல்லிடு தலைவரை ஒடசல் போ...,NOT
650,tam_test_651,"வாழ்த்துக்கள்,உண்மை கதை உலகத்துக்கு காட்டிய மோ...",NOT
651,tam_test_652,குருமா and பா..குஞ்சித் பயலுக கதர்ரது இன்னும் ...,NOT
652,tam_test_653,வெள்ளார்கள் சார்பாக படம் வெற்றி பெற வாழ்த்துக்...,NOT


In [ ]:
# Majority Voting

predictions = le.inverse_transform(mv_test_pred)

dataframe_test_mv = {
    'id': dataframe_test['id'],
    'text': dataframe_test['text'],
    'category': predictions
}

dataframe_test_mv = pd.DataFrame(dataframe_test_mv)
dataframe_test_mv.to_csv("Output/TF-IDF-MajorityVoting.csv")
dataframe_test_mv

,id,text,category
0,tam_test_1,தலைவா STR இதுக்குதான் கதுருந்தோம் மாஸ் தலைவா t...,NOT
1,tam_test_2,நாயுடு மக்கள் சார்பாக திரைப்படம் வெற்றி பெற வா...,NOT
2,tam_test_3,தில்லானா முயற்சி தஞ்சை கோனார்,NOT
3,tam_test_4,திரௌபதியின் துகிலுறித்த போது காத்த கண்ணனின் யா...,NOT
4,tam_test_5,நான் தியேட்டர்லே படம் பார்த்து 35 வருஷமாச்சு! ...,NOT
...,...,...,...
649,tam_test_650,நான் தலைவரோட ரசிகன்னு சொல்லிடு தலைவரை ஒடசல் போ...,NOT
650,tam_test_651,"வாழ்த்துக்கள்,உண்மை கதை உலகத்துக்கு காட்டிய மோ...",NOT
651,tam_test_652,குருமா and பா..குஞ்சித் பயலுக கதர்ரது இன்னும் ...,NOT
652,tam_test_653,வெள்ளார்கள் சார்பாக படம் வெற்றி பெற வாழ்த்துக்...,OFF
